Notebook for 
1. creating branched polymers
2. briefly visualizing them
3. saving them to .dat files for running in LAMMPS. 

By: Mihir Gowda



## Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import re
from scipy.spatial.transform import Rotation as R

## Generating 3D Polymers

#### Functions 

(feel free to just run this section and check out the examples to learn how to use quickly)

In [ ]:
starting_pos = np.array([np.array([1, 0 , 0])]) # x, y, z
deg = 30 # angle between consecutive atoms 
rad = np.radians(deg)

In [ ]:
def randunitv():
    randv = np.random.random(3)
    randvn = randv / np.sqrt(np.sum(randv**2))
    return np.array([randvn])

In [ ]:
def addalongx(point, number):
    points = [point]

    for n in range(number):
        if n % 2:
            y = -np.sin(rad)
        else: 
            y = np.sin(rad)
        points.append([((n+1) * np.cos(rad)) + point[0], y + point[1], point[2]])

    return points



In [ ]:
def addalongvector(point, number, vector, deg):
    points = [point]

    # rotate along x by deg
    rotvec = np.array([1, 0, 0])
    rad = np.radians(deg)

    rxexact = R.from_rotvec(rotvec)

    rxpos = R.from_rotvec(rad * rotvec)
    rxneg = R.from_rotvec(-rad * rotvec)
    rxpmat = rxpos.as_matrix()
    rxnmat = rxneg.as_matrix()

    addv = ((rxpmat @ vector.T).T)
    negv = ((rxnmat @ vector.T).T)

    addv /= np.linalg.norm(addv)
    negv /= np.linalg.norm(negv)
    vectorn = vector / np.linalg.norm(vector)

    for n in range(number):
        if n == 0 or deg == 0:
            points.append((vector + points[n])[0])
        else:
            points.append(((addv if n % 2 else negv) + points[n])[0])

    return np.array(points)

In [ ]:
def branchvectors(oldvec, numvectors):
    """
    Generate a nucleation point; a point where multiple branches spout from.

    Arguments:
    ----------
    oldvec: the incoming vector
    numvectors*: number of vectors that will branch from the point.
        *limited to 4 max currently
    """

    vectors = [oldvec]
    oldv = oldvec[0]

    # orthogonal vector, one dimension
    ov1 = np.random.randn(3)
    ov1 -= ov1.dot(oldv) * oldv
    ov1 /= np.linalg.norm(ov1)

    # other orthogonal vector, second dimension
    ov2 = np.cross(oldv, ov1)

    # choice of orthogonal vector to rotate around is randomized
    ovhere = (ov1 if np.round(np.random.random()) <= .5 else ov2)

    # rotations
    r120 = R.from_rotvec(np.radians(120) * ovhere).as_matrix()
    r109 = R.from_rotvec(np.radians(109.5) * ovhere).as_matrix()

    if numvectors == 2:
        vectors.append(-oldvec)

    if numvectors == 3:
        r1 = (r120 @ oldvec.T).T
        r2 = (r120 @ (r120 @ oldvec.T)).T
        vectors.append(r1)
        vectors.append(r2)

    if numvectors == 4:
        # one vector set 109.5 away from initial vector
        r1 = (r109 @ oldvec.T).T

        # need to rotate around the semicircle defined by first new vector
        r120circle = R.from_rotvec(np.radians(120) * oldvec[0]).as_matrix()
        r2 = (r120circle @ r1.T).T
        r3 = (r120circle @ (r120circle @ r1.T)).T

        vectors.append(r1)
        vectors.append(r2)
        vectors.append(r3)

    return np.array(vectors)[:, 0]



In [ ]:
def bondlistadd(init, length):
    return np.concatenate((np.array([np.arange(length) + init]).T, np.array([np.arange(length) + 1 + init]).T), 1)

In [ ]:
def bondlistaddsep(init, length, newcount):
    farray = np.array([np.arange(length) + newcount])
    farray[0][0] = init
    sarray = np.array([np.arange(length) + newcount + 1])
    return np.concatenate((farray.T, sarray.T), 1)


In [ ]:
def anglistadd(init, length):
    # length should be the branch length
    anglen = length - 1
    return np.concatenate((np.array([np.arange(anglen) + init]).T, 
                           np.array([np.arange(anglen) + init + 1]).T, 
                           np.array([np.arange(anglen) + init + 2]).T), 1)

In [ ]:
def anglistaddsep(init, length, newcount):
    length = length - 1
    farray = np.array([np.arange(length) + newcount])
    farray[0][0] = init
    sarray = np.array([np.arange(length) + newcount + 1])
    tarray = np.array([np.arange(length) + newcount + 2])
    return np.concatenate((farray.T, sarray.T, tarray.T), 1)

In [ ]:
def anglistfourway(catom, branchlen):
    # for a four branch junction (nucleus)
    atom1 = catom - 1
    atom2 = catom + 1
    atom3 = catom + 1 + branchlen
    atom4 = catom + 1 + 2*branchlen
    ang1 = np.array([[atom1, catom, atom2]])
    ang2 = np.array([[atom1, catom, atom3]])
    ang3 = np.array([[atom1, catom, atom4]])
    ang4 = np.array([[atom2, catom, atom3]])
    ang5 = np.array([[atom2, catom, atom4]])
    ang6 = np.array([[atom3, catom, atom4]])
    return np.concatenate((ang1.T, ang2.T, ang3.T, ang4.T, ang5.T, ang6.T), 1).T

In [ ]:
def anglistthreeway(catom, branchlen):
    # for a three branch junction (nucleus)
    atom1 = catom - 1
    atom2 = catom + 1
    atom3 = catom + 1 + branchlen
    ang1 = np.array([[atom1, catom, atom2]])
    ang2 = np.array([[atom1, catom, atom3]])
    ang3 = np.array([[atom2, catom, atom3]])
    return np.concatenate((ang1.T, ang2.T, ang3.T), 1).T

In [ ]:
def createmolecule(backbonelen,
                  numbernuc = 0,
                  branchlen = 0,
                  branchespernuc = 0,
                  degreevar = 0
                  ):
    """
    Create a random molecule!

    Arguments
    ---------
    backbonelen: Length of the backbone.
    numbernuc: Number of nucleation sites. These will be split along the backbone, and so will always be centered.
    branchlen: Length of branches. These only pertain to non-backbone branches.
    branchespernuc: Number of branches per nucleation site. Only applicable values are 3 and 4 currently. 
    degreevar: The degree of angle between consecutive atoms along the chain. Wouldn't advise above 45. 

    Returns
    --------
    (mpoints, bondlist, anglist)

    mpoints: 2-dimensional array of all atom positions
    bondlist: 2-dimensional array of all connections, ie. a list of [1, 2] indicates atom 1 is bonded to atom 2. 
    anglist: 2-dimensional array of all angles, ie. a list of [1, 2, 3] indicates atom 1, 2, and 3 are in succession.

    """
    starting_pos = np.array([[0, 0, 0]])
    bondlist = []
    bondcounter = 1
    mpoints = [starting_pos]
    bb = backbonelen - 1
    branchlen = int(branchlen)

    # If just a linear molecule, skip rest of code!
    if not numbernuc and not branchlen and not branchespernuc:
        mpoints = addalongvector(list(starting_pos[0]), bb, np.array([[0, 1, 0]]), degreevar)
        bondlist.append(np.concatenate((np.array([np.arange(bb) + 1]).T, np.array([np.arange(bb) + 2]).T), 1))
        anglist = anglistadd(1, bb)
        return mpoints, bondlist[0], anglist

    # Molecule segment calculations
    bbsegmentlen = int(np.round(backbonelen / (numbernuc + 1)))
    bbsegments = int(np.round(backbonelen / bbsegmentlen))
    bbsegs = bbsegmentlen * bbsegments
    calbackbonelen = bbsegs + numbernuc
    outerbblen = bbsegmentlen

    # Adjust the tails of the backbone, because can't assume constant backbone segment length
    if bbsegs > backbonelen:
        outerbblen -= 1

    # Notify behind the scenes calculations
    print(f"Rounded number of backbone segments to {bbsegments}")
    print(f"Initial backbone length is {calbackbonelen}")
    print(f"If needed, fixed backbone length to {outerbblen * bbsegments + numbernuc}")

    # Add the first outer segment of the backbone, until the first nucleus
    mpoints = (addalongvector(starting_pos[0], outerbblen, np.array([[0, 0, 1]]), degreevar))
    bondlist = bondlistadd(1, outerbblen)
    anglist = anglistadd(1, outerbblen)
    bondcounter += outerbblen

    # Iterate through the number of backbone segments, to add the branches and backbone
    # segments one chunk at a time
    for b in range(bbsegments - 1):
        vs = branchvectors(np.array([-(mpoints[-1] - mpoints[-2])]), branchespernuc)
        originalbond = bondcounter
        vsrelevant = vs[1:-1]
        bbchoice = vs[-1]
        center = mpoints[-1]

        # Build the branches
        for v in vsrelevant:
            mpoints = np.append(mpoints, (addalongvector(center, int(branchlen), np.array([v]), degreevar))[1:], axis = 0)
            bondlist = np.concatenate((bondlist, bondlistaddsep(originalbond, branchlen, bondcounter)), 0)
            # Add all angles except at the intersection
            anglist = np.concatenate((anglist, anglistaddsep(originalbond, branchlen, bondcounter)), 0)
            bondcounter += branchlen 

        # Build the backbone segment to connect to next nucleus, or the ending backbone segment
        effectivelen = bbsegmentlen if b != bbsegments-2 else outerbblen
        mpoints = np.append(mpoints, (addalongvector(center, effectivelen, np.array([bbchoice]), degreevar))[1:], axis = 0)
        bondlist = np.concatenate((bondlist, bondlistaddsep(originalbond, effectivelen, bondcounter)), 0)
        anglist = np.concatenate((anglist, anglistaddsep(originalbond, effectivelen, bondcounter)), 0)

        # Add the intersection angles
        if branchespernuc == 3:
            anglist = np.concatenate((anglist, anglistthreeway(originalbond, effectivelen)), 0)
        if branchespernuc == 4:
            anglist = np.concatenate((anglist, anglistfourway(originalbond, effectivelen)), 0)

        bondcounter += effectivelen

    bondlist = np.unique(bondlist, axis=0).astype(int)
    anglist = np.unique(anglist, axis=0).astype(int)

    return mpoints, bondlist, anglist



In [ ]:
def branchedmoltodat(atoms, bonds, angles, fname):
    """
    Saves a branched molecule generated by {createmolecule} to a file

    Arguments:
    -----------
    atoms: 2D array of atom positions.
    bonds: 2D array of bonds
    fname: Name of file*.
    *following this set of rules for naming:
        'bb{backbone length}nuc{number of nucleation sites}bl{length of branches}bn{number of branches}d{degrees of variation}.dat'
    
    Output:
    -------
    Written file.
    none.

    """
    natoms = len(atoms)
    nbonds = len(bonds)
    nangs = len(angles)

    # edit this variable to set save directory differently
    fpath = ""

    # couple of strict formatting for LAMMPS .dat formats
    wlist = [fname, '\n\n', 
             f'{natoms} atoms\n', 
             f'{nbonds} bonds\n', 
            f'{nangs} angles\n\n', 
             '1 atom types\n', 
            '1 bond types\n', 
             '1 angle types\n\n',
            '-1000 1000 xlo xhi\n', 
             '-1000 1000 ylo yhi\n',
            '-1000 1000 zlo zhi\n\n', 
             'Masses\n\n',
            '1 1\n\n', 
             'Atoms\n\n']

    # open and start editing file
    with open(f'{fpath}{fname}.dat', 'w+') as w:
        w.writelines(wlist)

        # writing the atoms coordinates
        for q, arr in enumerate(atoms):
            x = arr[0]
            y = arr[1]
            z = arr[2]
            line = f'{q+1}\t1\t1\t{x}\t{y}\t{z}\t0\t0\t0\n'
            w.write(line)

        w.write('\nBonds\n\n')
        # writing the bonds
        for q, barr in enumerate(bonds):
            line = f'{q+1}\t1\t{barr[0]}\t{barr[1]}\n'
            w.write(line)

        w.write('\nAngles\n\n')
        # writing the angles
        for q, aarr in enumerate(angles):
            line = f'{q+1}\t1\t{aarr[0]}\t{aarr[1]}\t{aarr[2]}\n'
            w.write(line)   



#### Examples

Assuming an *even distribution* of monomers across all branches (and backbone segments) throughout the molecule, molecules should be created as so:

In [ ]:
## Total Monomer count
a = 200

In [ ]:
## Linear 

x = a
testps, testbs, testas = createmolecule(a, 0, 0, 0, 30)

In [ ]:
## Plus-shaped

x = a/2 + 1 # branch segment length
testps, testbs, testas = createmolecule(x, 1, (x-1)/2, 4, 30)

In [ ]:
## T-shaped

x = a/3 * 2 
testps, testbs, testas = createmolecule(x, 1, x/2, 3, 30)

In [ ]:
## 3-Nucleation 4-Branched

x = a/10 * 4
testps, testbs, testas = createmolecule(x, 3, x/4, 4, 30)

In [ ]:
## 5-Nucleation 4-Branched

x = a/16 * 6
testps, testbs, testas = createmolecule(x, 5, x/6, 4, 30)

In [ ]:
# Visualize!

mol = testps

fig = px.scatter_3d(x=mol[:, 0], y=mol[:, 1], z=mol[:, 2], 
                    color = np.arange(len(mol)))
fig.show()

##### Mass-production of all polymers used in the thesis

Creating each shape of (Linear, Plus, T, N3B4, N5B4) for monomer amounts [100, 1200] at 100 intervals. 

In [ ]:
# Creating Linear
atomtotals = np.linspace(100, 1200, 12)
for a in atomtotals:
    # Get the monomer total (ie, 100, 200, .. 1200)
    a = int(a)

    # Create the molecule with Linear arrangement
    testps, testbs, testas = createmolecule(a, 0, 0, 0, 30)

    # Save all the information to a .dat file following nomenclature described in thesis
    branchedmoltodat(testps, testbs, testas, f'bb{a}nuc0bl0bn0d30tot{a}')

    # Print title of file
    print(f'bb{a}nuc0bl0bn0d30tot{a}')

In [ ]:
# Creating Plus-shaped
atomtotals = np.linspace(100, 1200, 12)
for a in atomtotals:
    # Get the monomer total (ie, 100, 200, .. 1200)
    a = int(a)

    # Extract backbone length from monomer total
    x = int(a/2 + 1)

    # Extract branch length from monomer total
    bl = int((x-1)/2)
    
    testps, testbs, testas = createmolecule(x, 1, bl, 4, 30)
    branchedmoltodat(testps, testbs, testas, f'bb{x}nuc1bl{bl}bn4d30tot{a}')
    print(f'bb{x}nuc1bl{bl}bn4d30tot{a}')

In [ ]:
# Creating T-shaped
atomtotals = np.linspace(100, 1200, 12)
for a in atomtotals:
    a = int(a)
    x = int(a/3 * 2)
    bl = int(x/2)
    testps, testbs, testas = createmolecule(x, 1, bl, 3, 30)
    branchedmoltodat(testps, testbs, testas, f'bb{x}nuc1bl{bl}bn3d30tot{a}')
    print(f'bb{x}nuc1bl{bl}bn3d30tot{a}')

In [ ]:
# Creating 3-Nucleation 4-Branched
atomtotals = np.linspace(100, 1200, 12)
for a in atomtotals:
    a = int(a)
    x = int(a/10 * 4)
    bl = int(x/4)
    testps, testbs, testas = createmolecule(x, 3, bl, 4, 30)
    branchedmoltodat(testps, testbs, testas, f'bb{x}nuc3bl{bl}bn4d30tot{a}')
    print(f'bb{x}nuc3bl{bl}bn4d30tot{a}')

In [ ]:
# Creating 5-Nucleation 4-Branched
atomtotals = np.linspace(100, 1200, 12)
for a in atomtotals:
    a = int(a)
    x = int(a/16 * 6)
    bl = int(x/6)
    testps, testbs, testas = createmolecule(x, 5, bl, 4, 30)
    branchedmoltodat(testps, testbs, testas, f'bb{x}nuc5bl{bl}bn4d30tot{a}')
    print(f'bb{x}nuc5bl{bl}bn4d30tot{a}')